In [ ]:
#import packages
import cv2
import io
import os
import json
import gc
import pandas as pd
import re
import numpy as np
import math
import matplotlib.pyplot as plt 
from matplotlib.animation import FuncAnimation
from matplotlib.colors import ListedColormap
from PIL import Image
from tqdm import tqdm

In [ ]:
# === USER-DEFINED PARAMETERS ===
ppt = "YOUR_PPT_ID"
body_part = "_all"
fps = 30
interval_in_secs = 1

# File paths (replace with your paths)
data_file = f"/path/to/data/new_df_10_mins_{ppt}_latest.pkl"
video_input_path = f"/path/to/videos/{ppt}_video.mp4"

# Output folders for visualization (replace with your output directories)
output_folder = f"/path/to/output/{ppt}_visualisation"
videoframes_path = os.path.join(output_folder, "videoframes")
rawskplotted_path = os.path.join(output_folder, "rawskplotted")
rawcombined_path = os.path.join(output_folder, "rawcombined")
correctedskplotted_path = os.path.join(output_folder, "correctedskplotted")
correctedcombined_path = os.path.join(output_folder, "correctedcombined")

# Create folders if they don't exist
for folder in [videoframes_path, rawskplotted_path, rawcombined_path, correctedskplotted_path, correctedcombined_path]:
    if not os.path.exists(folder):
        os.makedirs(folder)

# Function to get video dimensions
def get_video_dimensions(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise IOError(f"Cannot open video file {video_path}")
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    cap.release()
    return width, height

# Get video dimensions
try:
    x_dimension, y_dimension = get_video_dimensions(video_input_path)
    print(f"Video dimensions: {x_dimension} x {y_dimension}")
except IOError as e:
    print(e)
    x_dimension, y_dimension = None, None

# Determine aspect ratio
if (x_dimension, y_dimension) in [(1920, 1080), (1280, 720)]:
    x_aspect, y_aspect = 16, 9
elif (x_dimension, y_dimension) == (1920, 1072):
    x_aspect, y_aspect = 120, 67
elif (x_dimension, y_dimension) == (1920, 1440):
    x_aspect, y_aspect = 4, 3
elif (x_dimension, y_dimension) == (1440, 1080):
    x_aspect, y_aspect = 4, 3
else:
    x_aspect, y_aspect = None, None

# Load dataframe
df = pd.read_pickle(data_file)

# Define first and last frame numbers
first_frame = df.index[0]
last_frame = first_frame + 10 * 60 * fps  # example: 10 minutes segment

# Convert frame to time (minutes and seconds)
def frame_to_time(frame_number, fps):
    total_seconds = frame_number / fps
    minutes = int(total_seconds // 60)
    seconds = int(total_seconds % 60)
    return minutes, seconds

first_min, first_sec = frame_to_time(first_frame, fps)
last_min, last_sec = frame_to_time(last_frame, fps)

print(f"First Frame: {first_frame} -> Time: {first_min}m {first_sec}s")
print(f"Last Frame: {last_frame} -> Time: {last_min}m {last_sec}s")

# Reset index and reorder columns
df['frame'] = df.index
df = df.reset_index(drop=True)
new_col_order = ['frame'] + [col for col in df.columns if col != 'frame']
df = df[new_col_order]

# Select only key columns and melt to long format
df_to_melt = df[['frame', 'P1', 'P2', 'P3']]

def prepare_long_df(df_subset, person_id):
    df_tmp = df_subset[['frame', person_id]].copy()
    df_tmp['id'] = person_id
    df_tmp.columns = ['frame', 'coordinates', 'id']
    return df_tmp

P1 = prepare_long_df(df_to_melt, 'P1')
P2 = prepare_long_df(df_to_melt, 'P2')
P3 = prepare_long_df(df_to_melt, 'P3')

df_long = pd.concat([P1, P2, P3], ignore_index=True)

print(df_long.head())

In [ ]:

def convert_coordinates(value):
    try:
        # If the value is already a NumPy array, return it
        if isinstance(value, np.ndarray):
            return value
        # If the value is a float, return a NumPy array with a single value
        elif isinstance(value, float):
            return np.array([value])
        # Try to load the value as JSON and convert to a NumPy array
        else:
            return np.array(json.loads(value))
    except (json.JSONDecodeError, ValueError):
        # If it fails, return None
        return None

def plot_coordinates_frames(df, first_min, first_sec, last_min, last_sec, fps, interval_in_secs, output_prefix, output_path):
    # Convert start and end timepoints to frames
    first_frame = (first_min * 60 + first_sec) * fps
    last_frame = (last_min * 60 + last_sec) * fps

    df['id'] = df['id'].astype('category')

    cmap = ListedColormap(['green', 'red', 'blue'])
    id_to_color = {'P1': 'green', 'P2': 'red', 'P3': 'blue'}

    # Define the frame numbers to plot
    frame_numbers = range(first_frame, last_frame + 1, int(fps * interval_in_secs))

    # Loop through the frame numbers and plot the data for each person
    for frame in tqdm(frame_numbers, desc='Processing Frames'):
        # Filter the data to select only the rows for the current frame
        frame_data = df[df['frame'] == frame].copy()

        # Convert string representation of arrays to actual NumPy arrays
        frame_data['coordinates'] = frame_data['coordinates'].apply(convert_coordinates)

        # Plot the data for the current frame
        plt.figure(figsize=(x_dimension, y_dimension), facecolor='none')
        fig, ax = plt.subplots()

        for index, row in frame_data.dropna(subset=['coordinates']).iterrows():
            coordinates = row['coordinates']

            # Check if the coordinates is a NumPy array
            if isinstance(coordinates, np.ndarray):
                # Extract x and y coordinates from the array
                if coordinates.ndim == 1:
                    # If 1D, assume it's a single point
                    x_coords, y_coords = coordinates[0], coordinates[0]
                else:
                    # If 2D, extract x and y coordinates
                    x_coords, y_coords = coordinates[:, 0], coordinates[:, 1]

                # Plot the points for each person - S IS MARKER SIZE
                ax.scatter(x_coords, y_coords, c=id_to_color[row['id']], s=50, alpha=1)

        # Set axis and label properties
        ax = plt.subplot(1, 1, 1)
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_xlabel('')
        ax.set_ylabel('')
        ax.set_xlim([0, x_dimension])
        ax.set_ylim([0, y_dimension])
        ax.set_facecolor('none')
        ax.axis('off')

        # Flip horizontally and rotate by 180 degrees
        ax.set_xlim(ax.get_xlim()[::-1])
        ax.set_xticks(ax.get_xticks()[::-1])
        ax.invert_xaxis()
        ax.invert_yaxis()

        # Remove all margins
        fig.subplots_adjust(left=0, right=1, bottom=0, top=1)

        # Save the plot to a file with the current frame number
        fig.savefig(f'frame_{frame}.png', dpi=96, transparent=True)

        # Open the saved figure using Pillow
        image = Image.open(f'frame_{frame}.png')

        # Resize the image to the desired dimensions
        image = image.resize((x_dimension, y_dimension))

        # Save the resized image
        image.save(f"{output_path}/{output_prefix}_{frame}.png", transparent=True)

        # Close the plot to release memory
        plt.close()

        # Delete the temporary frame image
        os.remove(f'frame_{frame}.png')

def extract_frames(video_input_path, output_dir, first_min, first_sec, last_min, last_sec, fps, interval_in_secs, output_prefix, quality, x_dimension, y_dimension, resize=False):
    # Function code here

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    first_frame = (first_min * 60 + first_sec) * fps
    last_frame = (last_min * 60 + last_sec) * fps
    frame_interval = int(fps * interval_in_secs)

    cap = cv2.VideoCapture(video_input_path)
    num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    video_duration = num_frames / fps

    frame_count = 0
    fractional_frame_count = 0

    while True:
        ret, frame = cap.read()

        if not ret:
            break

        if first_frame <= frame_count <= last_frame and frame_count % frame_interval == 0:
            output_path = os.path.join(output_dir, f"{output_prefix}_{frame_count}.jpg")
            
            if resize and x_dimension is not None and y_dimension is not None:
                frame = cv2.resize(frame, (x_dimension, y_dimension))

            cv2.imwrite(output_path, frame, [int(cv2.IMWRITE_JPEG_QUALITY), quality])

        frame_count += 1
        fractional_frame_count = frame_count / fps

        if frame_count > last_frame:
            break

    cap.release()
    cv2.destroyAllWindows()

def combine_images(folder1_path, folder2_path, combined_path, output_prefix, max_width=200, max_height=75, quality=1):
    # Get the list of image file names in both folders
    folder1_images = os.listdir(folder1_path)
    if '.DS_Store' in folder1_images:
        folder1_images.remove('.DS_Store')

    folder2_images = os.listdir(folder2_path)
    if '.DS_Store' in folder2_images:
        folder2_images.remove('.DS_Store')

    # Define a regular expression pattern to extract the number from the image filename
    number_pattern = re.compile(r"(?<![0-9])\d+(?![0-9])")
    
    # Loop through the image files in folder1
    for image1_file in folder1_images:
        # Extract the number from the filename of the image in folder1
        number1 = int(number_pattern.search(image1_file).group())

        # Loop through the image files in folder2
        for image2_file in folder2_images:
            # Extract the number from the filename of the image in folder2
            number2 = int(number_pattern.search(image2_file).group())

            if number1 == number2:
                # Load the images from both folders
                image1 = Image.open(f"{folder1_path}/{image1_file}")
                image2 = Image.open(f"{folder2_path}/{image2_file}")

                # Resize the images to a smaller resolution
                image1.thumbnail((max_width, max_height))
                image2.thumbnail((max_width, max_height))

                img1 = image1.convert("RGBA")
                img2 = image2.convert("RGBA")

                combined_img = Image.alpha_composite(img1, img2)

                # Save the combined image to disk as a PNG file with the desired quality level
                combined_img.save(f"{combined_path}/{output_prefix}_{image1_file}.png", quality=quality)

                # Reduce the dimensions of the image
                with Image.open(f"{combined_path}/{output_prefix}_{image1_file}.png") as im:
                    x, y = im.size
                    if x > max_width or y > max_height:
                        ratio = min(max_width / x, max_height / y)
                        new_x = int(x * ratio)
                        new_y = int(y * ratio)
                        im_resized = im.resize((new_x, new_y))
                        im_resized.save(f"{combined_path}/{output_prefix}_{image1_file}")

                break        

In [ ]:
plot_coordinates_frames(first_min = 0, first_sec = 42, last_min = 0, last_sec = 43,
               df=df_long, fps=fps, interval_in_secs=1/fps, output_prefix='output', output_path=rawskplotted_path)

In [ ]:
video_input_path = video_input_path
output_dir = videoframes_path
first_min, first_sec = 0, 42
last_min, last_sec = 0, 43
fps = fps
interval_in_secs = 1/fps
output_prefix = "videoframe"
x_dimension = x_dimension
y_dimension = y_dimension
quality = 1

#extract_frames(video_input_path, output_dir, first_min, first_sec, last_min, last_sec, fps, interval_in_secs, output_prefix, quality=1)
extract_frames(video_input_path, output_dir, first_min, first_sec, last_min, last_sec, fps, interval_in_secs, output_prefix, quality, x_dimension, y_dimension, resize=True)

In [ ]:
folder1_path = videoframes_path
folder2_path = rawskplotted_path
combined_path = rawcombined_path_2
output_prefix = "combined_"

combine_images(folder1_path, folder2_path, combined_path, output_prefix, max_width=200, max_height=75, quality=1)

In [ ]:
df_long = df_long[["frame", "coordinates", "id"]]
df_long = df_long.reset_index(drop=True)

df_copy = df_long.copy()

In [ ]:
# Define the frames to modify 
frames_to_modify = list(range(0, 1276))
ids_to_modify = ['P1', 'P2', 'P3']

# Loop over each row and modify the 'id' column if the criteria are met
for index, row in df_copy.iterrows():
    if row['frame'] in frames_to_modify:
        if row['id'] == 'P1':
            df_copy.loc[index, 'id'] = 'P3'
            print(f"Frame {row['frame']} modified to P3")
        elif row['id'] == 'P2':
            df_copy.loc[index, 'id'] = 'P1'
            print(f"Frame {row['frame']} modified to P1")
        elif row['id'] == 'P3':
            df_copy.loc[index, 'id'] = 'P2'
            print(f"Frame {row['frame']} modified to P2")

In [ ]:
#check differences between original and corrected. it can be inferred that self refers to a DataFrame object (df) on which the compare() method is being called, while other refers to another DataFrame object (df_copy) that is being compared against self (i.e., df).
diff = df_long.compare(df_copy)
print(diff)

In [ ]:
df_corrected.to_pickle(os.path.join(final_corrected_file_path, "full_corrected_df_latest_"+str(ppt)+".pkl"))
df_corrected.to_csv(os.path.join(final_corrected_file_path, "full_corrected_df_latest_"+str(ppt)+".csv"))